In [1]:
import h5py
import warnings
import sys 
if not sys.warnoptions:
    warnings.simplefilter("ignore")
import os 
import glob
import time
from copy import deepcopy
import numpy as np
import pandas as pd 

from nilearn import datasets, image
from nilearn import surface
from nilearn import plotting
from nilearn.input_data import NiftiMasker, NiftiLabelsMasker
import nibabel as nib

from brainiak import image, io
from brainiak.isc import isc, isfc, permutation_isc
from brainiak.isc import compute_summary_statistic
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d 
import seaborn as sns 
import pandas as pd
from importlib import reload 
import scipy.io as sio

# import own functions
import utils
reload(utils)
from utils import decode_variable, get_in_shape_levels, get_in_shape_blocks, plot_sub_isc_statmap, plot_statmap_onesub
from utils import plot_statmap_avg, surface_plot_avg, test_plot_statmap_avg

%autosave 30
%matplotlib inline
sns.set(style = 'white', context='talk', font_scale=1, rc={"lines.linewidth": 2})

Autosaving every 30 seconds


In [2]:
# specify local path
path = '/Users/Daphne/data/'

# parameters
num_subjects = 8

# load all bold data
games_betas = np.load(path+'bold_data_games.npy')
blocks_betas = np.load(path+'bold_data_blocks.npy')
levels_betas = np.load(path+'bold_data_levels.npy')
# boxcar_betas = np.load(path+'bold_data_boxcar.npy')

# load mask and get voxel coordinates
mask_arr = np.load(path+'mask_arr.npy') # all masks are the same
whole_brain_mask = mask_arr[0] # so we can pick any one from the array
coords = np.where(whole_brain_mask == 1) # so need one set of voxel coordinates for all

# Save

In [24]:
#  --- choose type of data ---
betas = levels_betas

In [25]:
from utils import isc_onesub, array_correlation, get_significant_corrs, compute_avg_iscs

In [26]:
iscs_r = []
iscs_pvalues = []

# iterate through each subject
for s in range(num_subjects): 

    iscs_r_sub, iscs_p_sub = isc_onesub(betas, s) 
    
    # store subject data
    iscs_r.append(iscs_r_sub)
    iscs_pvalues.append(iscs_p_sub)
    print(s)

# convert to numpy arrays
iscs_r_arr = np.array(iscs_r)
iscs_pvalues_arr = np.array(iscs_pvalues)

# get the significant values 
significant_r_coeffs = get_significant_corrs(iscs_r_arr=iscs_r_arr, iscs_pvalues_arr=iscs_pvalues_arr, alpha=0.05)
print(significant_r_coeffs)

0
1
2
3
4
5
6
7
[[ 0.          0.          0.         ...  0.          0.
   0.        ]
 [ 0.          0.          0.         ...  0.28666462  0.27016796
   0.        ]
 [ 0.40445141  0.44137352  0.31407147 ...  0.          0.
   0.        ]
 ...
 [ 0.          0.          0.         ...  0.          0.
   0.        ]
 [-0.3093235  -0.3208494  -0.26808857 ...  0.          0.
   0.        ]
 [ 0.          0.          0.         ...  0.          0.
   0.        ]]


In [27]:
# collapse iscs (also with utils)
collapsed_isc_corrs = compute_avg_iscs(iscs_r_arr)

Save:

- iscs_r_arr
- iscs_pvalues_arr
- significant_r_coeffs
- collapsed_isc_corrs

In [28]:
sio.savemat('iscs_r_levels.mat', {'mydata': iscs_r_arr})

In [29]:
sio.savemat('iscs_p_levels.mat', {'mydata': iscs_pvalues_arr})

In [30]:
sio.savemat('significant_r_levels.mat', {'mydata': significant_r_coeffs})

In [31]:
sio.savemat('collapsed_iscs_levels.mat', {'mydata': collapsed_isc_corrs})

# Check if the outcomes are equal

In [32]:
# compute the isc correlations using the leave one out approach
isc_maps = isc(betas, pairwise=False)

# compute the average across participants with brainiak
avg_isc_corrs = isc(betas, pairwise=False, summary_statistic='mean', tolerate_nans=True) 

In [33]:
isc_maps

array([[-0.08647298, -0.08264689,  0.05209291, ..., -0.04728233,
        -0.02028276, -0.0585491 ],
       [ 0.01931597,  0.04682847, -0.04335183, ...,  0.28666462,
         0.27016796, -0.18529388],
       [ 0.40445141,  0.44137352,  0.31407147, ...,  0.23743184,
         0.15753744, -0.14028751],
       ...,
       [ 0.1656714 ,  0.11414849,  0.03646999, ..., -0.11111457,
        -0.11722488,  0.23958578],
       [-0.3093235 , -0.3208494 , -0.26808857, ...,  0.04332081,
         0.14445493,  0.03816058],
       [ 0.02986205,  0.04564478, -0.09362682, ...,  0.18391026,
         0.20258632, -0.23755461]])

In [34]:
iscs_r_arr

array([[-0.08647298, -0.08264689,  0.05209291, ..., -0.04728233,
        -0.02028276, -0.0585491 ],
       [ 0.01931597,  0.04682847, -0.04335183, ...,  0.28666462,
         0.27016796, -0.18529388],
       [ 0.40445141,  0.44137352,  0.31407147, ...,  0.23743184,
         0.15753744, -0.14028751],
       ...,
       [ 0.1656714 ,  0.11414849,  0.03646999, ..., -0.11111457,
        -0.11722488,  0.23958578],
       [-0.3093235 , -0.3208494 , -0.26808857, ...,  0.04332081,
         0.14445493,  0.03816058],
       [ 0.02986205,  0.04564478, -0.09362682, ...,  0.18391026,
         0.20258632, -0.23755461]])

In [35]:
np.allclose(isc_maps, iscs_r_arr) # Returns True if two arrays are element-wise equal within a tolerance.

True

In [36]:
collapsed_isc_corrs # from utils

array([ 0.01996267,  0.02655487, -0.04098616, ...,  0.10641914,
        0.12673402, -0.01467338])

In [37]:
avg_isc_corrs # from brainiak

array([ 0.01996267,  0.02655487, -0.04098616, ...,  0.10641914,
        0.12673402, -0.01467338])

In [38]:
np.allclose(collapsed_isc_corrs, avg_isc_corrs)

True